# Project: Cohort Analysis

### Calculate the days between first and second purchase

In [0]:
CREATE OR Replace TABLE workspace.bigquery_db_cohort_db.cohort_analysis AS

-- Calculate the first Purchase Date for each customer

with first_purchase_table as ( 
  SELECT customer_id, 
        min(order_date) as first_purchase_date,
        count(order_id) as total_orders
  From workspace.bigquery_db_cohort_db.ecom_orders
  group by customer_id
),

-- Calculate the second purchase date for each customer
second_purchase_table as (
  SELECT e.customer_id, 
    min(e.order_date) as second_purchase_date
  From workspace.bigquery_db_cohort_db.ecom_orders e
  left join first_purchase_table f
  on e.customer_id = f.customer_id
  WHERE order_date > f.first_purchase_date
  group by e.customer_id
),

-- Combine both tables & calculate days between purchases
final_result as (
  SELECT  
  f.customer_id, 
  f.first_purchase_date, 
  s.second_purchase_date,
  f.total_orders,
  datediff(s.second_purchase_date, f.first_purchase_date) as days_between_purchases
from first_purchase_table f
left join second_purchase_table s
on f.customer_id = s.customer_id
)

-- Create the final table

SELECT *
FROM final_result;

### Show the final table

In [0]:
select * from workspace.bigquery_db_cohort_db.cohort_analysis
limit 10